### Preprocessing  
In this file I prepare the data for exploration by cleaning missing values, simplifying catagories, and dropping anything unecessary. This notebook uses a datafile that is too large for github and so while I wanted to include my process it should not be run.

In [1]:
import pandas as pd 
import numpy as np 
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('okcupid_profiles.csv')   
kc = ['age', 'sex', 'orientation', 'body_type', 'diet', 'drinks', 'drugs', 'education', 'ethnicity', 'height', 'job', 'location', 
      'offspring', 'pets', 'religion', 'sign','smokes'] 
data = data[kc]

In [3]:
data.head(3)

,age,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,height,job,location,offspring,pets,religion,sign,smokes
0,22,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",75.0,transportation,"south san francisco, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,gemini,sometimes
1,35,m,straight,average,mostly other,often,sometimes,working on space camp,white,70.0,hospitality / travel,"oakland, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,cancer,no
2,38,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,68.0,NaN,"san francisco, california",NaN,has cats,NaN,pisces but it doesn&rsquo;t matter,no


In [4]:
data.job.value_counts() 
#Because the is going to be my target variable/ what my classes are based on I will start by recoving the categories I will not be 
#including and creating a new column that aligns with with one of the four categories I will be categorizing them into   

other                                7589
student                              4882
science / tech / engineering         4848
computer / hardware / software       4709
artistic / musical / writer          4439
sales / marketing / biz dev          4391
medicine / health                    3680
education / academia                 3513
executive / management               2373
banking / financial / real estate    2266
entertainment / media                2250
law / legal services                 1381
hospitality / travel                 1364
construction / craftsmanship         1021
clerical / administrative             805
political / government                708
rather not say                        436
transportation                        366
unemployed                            273
retired                               250
military                              204
Name: job, dtype: int64

In [5]:
jobs_being_used = ['transportation', 'hospitality / travel', 'artistic / musical / writer', 'computer / hardware / software',
       'banking / financial / real estate', 'entertainment / media','sales / marketing / biz dev','medicine / health',
       'science / tech / engineering', 'executive / management','education / academia', 'clerical / administrative',
       'construction / craftsmanship', 'political / government', 'law / legal services','military'] 
df = data[data['job'].isin(jobs_being_used)]

In [6]:
df.job.value_counts()

science / tech / engineering         4848
computer / hardware / software       4709
artistic / musical / writer          4439
sales / marketing / biz dev          4391
medicine / health                    3680
education / academia                 3513
executive / management               2373
banking / financial / real estate    2266
entertainment / media                2250
law / legal services                 1381
hospitality / travel                 1364
construction / craftsmanship         1021
clerical / administrative             805
political / government                708
transportation                        366
military                              204
Name: job, dtype: int64

In [7]:
#Creating another column that identies wether person if above avarage height for thier sex or not 
df['job_catagories'] = np.nan 
df['job_catagories'] = df['job'] 
df.job_catagories.replace(['computer / hardware / software','science / tech / engineering'
                     ],'science/tech', inplace=True)  
df.job_catagories.replace(['transportation', 'hospitality / travel','medicine / health', 'education / academia','political / government',
                'law / legal services', 'military'
                     ],'public_services', inplace=True)  
df.job_catagories.replace(['artistic / musical / writer','entertainment / media','construction / craftsmanship'
                     ],'entertainment/artist/craftmans', inplace=True)  
df.job_catagories.replace(['banking / financial / real estate','sales / marketing / biz dev','executive / management', 
                'clerical / administrative'],'other_white_collar', inplace=True)  


In [8]:
df['job_catagories'].value_counts()

public_services                   11216
other_white_collar                 9835
science/tech                       9557
entertainment/artist/craftmans     7710
Name: job_catagories, dtype: int64

In [9]:
df['job_catagories'].unique()

array(['public_services', 'entertainment/artist/craftmans',
       'science/tech', 'other_white_collar'], dtype=object)

In [10]:
df = df[df['sign'].notna()]

In [11]:
len(df)

32144

In [12]:
df.isna().sum()

age                   0
sex                   0
orientation           0
body_type          2266
diet              11937
drinks              798
drugs              7411
education          1988
ethnicity          2262
height                0
job                   0
location              0
offspring         17789
pets               8473
religion           8573
sign                  0
smokes             2103
job_catagories        0
dtype: int64

In [13]:
#df.status.value_counts()  
#From this data is probably makes the most sense to simply drop the unknown and possibly 
#the entire column since because the data came from a dating site this indicator is less 
#likely to be reasonably representative

In [14]:
df['sex'] = df['sex'].replace(['m','f'],['male','female'])
df.sex.value_counts() 


male      19795
female    12349
Name: sex, dtype: int64

In [18]:
df.orientation.value_counts() 
#This column is highly imblanced, I am choosign to combine the two minority classes into one and may rebalance this later one.

heterosexual        27863
not_heterosexual     4281
Name: orientation, dtype: int64

In [19]:
df['orientation'] = df['orientation'].replace(['straight','gay','bisexual'],
                                          ['heterosexual','not_heterosexual','not_heterosexual'])


In [22]:
df.body_type.value_counts() 
#This column has been reconfigured to group up similar descriptions. I have also changed the descriptions of "jacked" and "used up" 
#as they are desciptors with ambiguous meaning and changed nan and rather not say to average as well. 

body_fit/athletic          13725
body_average               10709
body_curvy/full_figured     4509
body_thin/skinny            3201
Name: body_type, dtype: int64

In [21]:
df['body_type'] = df['body_type'].replace(['fit','athletic','thin','skinny','curvy','a little extra', 'full figured', 'overweight',
                                          'jacked','used up', 'rather not say','average'],
                                          ['body_fit/athletic','body_fit/athletic','body_thin/skinny','body_thin/skinny', 
                                           'body_curvy/full_figured', 'body_curvy/full_figured','body_curvy/full_figured',
                                           'body_curvy/full_figured', 'body_average', 'body_average', 'body_average','body_average'])
df['body_type'] = df['body_type'].fillna('body_average')

In [25]:
df.diet.value_counts() 
#This column had a large number of missing values. I chose to include them in the anything catagory for the sake retaining as much 
#data as possible but also with the assumption that if wasn't important enough to fill in it is unlikely to be too relevant to the 
#observations life.

diet_anything            27961
diet_vegan/vegetarian     3056
diet_other                1127
Name: diet, dtype: int64

In [24]:
df['diet'] = df['diet'].replace(['strictly anything', 'mostly other', 'mostly anything', 'mostly vegetarian', 'strictly vegan', 
                                  'vegetarian','mostly halal', 'strictly vegetarian', 'strictly other', 'mostly kosher',
                                 'mostly vegan', 'vegan', 'strictly kosher', 'kosher', 'strictly halal', 'halal','other','anything'],
                                          ['diet_anything','diet_other','diet_anything','diet_vegan/vegetarian', 'diet_vegan/vegetarian', 'diet_vegan/vegetarian',
                                          'diet_other','diet_vegan/vegetarian', 'diet_other', 'diet_other', 'diet_vegan/vegetarian', 'diet_vegan/vegetarian','diet_other',
                                          'diet_other','diet_other','diet_other','diet_other','diet_anything'])
df['diet'] = df['diet'].fillna('diet_anything')

In [28]:
df.drinks.value_counts() 
#Simplified catagories 

drinks                      26578
drinks_rarely/not_at_all     4768
unknown_if_drinker            798
Name: drinks, dtype: int64

In [27]:
df['drinks'] = df['drinks'].replace(['socially', 'often', 'not at all', 'rarely','very often','desperately'],
                                    ['drinks', 'drinks','drinks_rarely/not_at_all','drinks_rarely/not_at_all','drinks','drinks' ])
df['drinks'] = df['drinks'].fillna('unknown_if_drinker')

In [30]:
df.drugs.value_counts() 
#Simplified catagories 

no_drugs            20400
unknown_if_drugs     7411
drugs                4333
Name: drugs, dtype: int64

In [29]:
df['drugs'] = df['drugs'].replace(['never', 'sometimes', 'often'],
                                ['no_drugs', 'drugs','drugs'])
df['drugs'] = df['drugs'].fillna('unknown_if_drugs')

In [32]:
df.education.value_counts() 
#Simplified the catagories including changing any "space camp" responses to unknown since it seems unlikely that many people went to 
#space camp 

college_graduate_or_higher    25174
no_college_degree              3993
unknown_education              2977
Name: education, dtype: int64

In [31]:
df.education.replace(['graduated from college/university','working on masters program','graduated from masters program', 
                      'college/university','graduated from law school', 'graduated from two-year college','working on med school',
                       'graduated from ph.d program','graduated from med school', 'two-year college', 'working on ph.d program',
                       'dropped out of ph.d program','dropped out of med school','working on law school',
                      'dropped out of masters program', 'masters program', 'ph.d program', 'law school',
                      'dropped out of law school', 'med school'], 'college_graduate_or_higher', inplace=True) 

df.education.replace(['working on college/university','dropped out of college/university','dropped out of high school',
                      'dropped out of two-year college', 'working on two-year college','high school', 'graduated from high school',
                      'working on high school'],'no_college_degree', inplace=True)  

df.education.replace(['working on space camp', 'graduated from space camp', 'dropped out of space camp',
                     'dropped out of space camp','space camp'
                     ],'unknown_education', inplace=True)   
df['education'] = df['education'].fillna('unknown_education')

In [34]:
df.describe()

,age,height
count,32144.000000,32144.000000
mean,33.479778,68.394102
std,9.229992,3.959041
min,18.000000,1.000000
25%,27.000000,66.000000
50%,31.000000,69.000000
75%,38.000000,71.000000
max,69.000000,95.000000


In [33]:
df.height.value_counts() 
#The column will be capped to reasonable nubers as descibed by this website: https://dqydj.com/height-percentile-calculator-for-men-and-women/ 
#According to this less than 1 percent of women are below 4'8" and less than 1 percent of men are taller than 6'3" with an std of about
#three inches inbetween 

70.0    3271
68.0    3061
72.0    2906
67.0    2866
69.0    2803
71.0    2629
66.0    2477
64.0    1983
65.0    1974
73.0    1589
74.0    1444
63.0    1433
62.0    1151
75.0     733
61.0     527
76.0     452
60.0     393
77.0     153
59.0      97
78.0      64
79.0      35
58.0      25
80.0      13
57.0       9
83.0       7
81.0       5
56.0       5
55.0       4
82.0       4
36.0       3
84.0       3
37.0       2
95.0       2
88.0       2
50.0       1
85.0       1
53.0       1
49.0       1
8.0        1
3.0        1
48.0       1
1.0        1
4.0        1
91.0       1
90.0       1
93.0       1
43.0       1
86.0       1
9.0        1
52.0       1
26.0       1
54.0       1
47.0       1
Name: height, dtype: int64

In [35]:
def cap_height(row):
    if row['height'] > 75.0:
        row['height'] = 79.0 #column average
    if row['height'] < 52:
        row['height'] = 56
    return row  
df = df.apply(cap_height, axis = 1)

In [36]:
#Creating another column that identies wether person if above avarage height for thier sex or not 
df['above_average_height'] = np.nan 
def aah_column(row): 
    male_avg = 69.2
    female_avg = 63.6
    if row['sex'] == 'm':
        if row['height'] > male_avg: 
            row['above_average_height'] = 'above_average_height' 
        else: 
            row['above_average_height'] = 'not_above_average_height' 
    else: 
        if row['height'] > female_avg: 
            row['above_average_height'] = 'above_average_height'  
        else: 
            row['above_average_height'] = 'not_above_average_height'
    return row 

df = df.apply(aah_column, axis = 1)

In [37]:
df.above_average_height.value_counts()

above_average_height        28481
not_above_average_height     3663
Name: above_average_height, dtype: int64

In [38]:
#data.income.value_counts() 
#Column will be dropped due to most not having answered this question and being that it is from a dating site, the chances of users
#inflating this number in my opinion is high

In [41]:
df.location.value_counts() 
#Since this data was taken from the san fran area most users will be in this area and so I have simplified the column to a binary 

san_francisco_ca     17394
not_san_francisco    14750
Name: location, dtype: int64

In [40]:
df['location'] = df['location'].fillna('not_san_francisco')
df.location.replace(['san francisco, california', 'south san francisco, california'],  'san_francisco_ca', inplace=True) 
#df['location'] = np.where(df['location'] in ['san francisco, california', 'south san francisco, california'], 'san_francisco_ca', 'not_san_francisco')
df.location.replace(['oakland, california', 'daly city, california','atherton, california', 
                     'san leandro, california', 'san rafael, california', 'walnut creek, california',  'berkeley, california', 
                     'belmont, california','san jose, california', 'palo alto, california', 'emeryville, california', 
                     'el granada, california', 'castro valley, california', 'fairfax, california','mountain view, california', 
                     'menlo park, california', 'burlingame, california', 'alameda, california','benicia, california', 
                     'mill valley, california', 'san mateo, california', 'redwood city, california', 'el cerrito, california', 
                     'stanford, california', 'san pablo, california', 'lafayette, california', 'fremont, california', 
                     'orinda, california', 'novato, california', 'vallejo, california', 'san lorenzo, california','san carlos, california', 'pacifica, california',
                     'hayward, california', 'foster city, california','hercules, california', 'bolinas, california',
                     'larkspur, california', 'moraga, california', 'albany, california','martinez, california', 'san bruno, california',
                     'millbrae, california', 'el sobrante, california','richmond, california', 'petaluma, california','pinole, california', 'pleasant hill, california',
                     'san geronimo, california', 'san anselmo, california','sausalito, california', 'crockett, california',
                    'boulder, colorado', 'half moon bay, california','belvedere tiburon, california', 'montara, california', 'corte madera, california', 'new york, new york',
                     'green brae, california', 'ross, california', 'east palo alto, california', 'brisbane, california',
                     'hacienda heights, california', 'point richmond, california','sacramento, california', 'rodeo, california', 'portland, oregon',
                       'tucson, arizona', 'honolulu, hawaii', 'billings, montana','west oakland, california', 'glencove, california',
                       'tiburon, california', 'peoria, illinois', 'bellwood, illinois',
                       'nha trang, vietnam', 'hillsborough, california','moss beach, california', 'kensington, california', 'kentfield, california', 'redwood shores, california',
                       'woodside, california', 'lagunitas, california','studio city, california', 'concord, california','forest knolls, california', 'edinburgh, united kingdom',
                       'london, united kingdom', 'chicago, illinois', 'colma, california','los angeles, california', 'south wellfleet, massachusetts', 'piedmont, california', 'los gatos, california', 'boise, idaho',
                       'islip terrace, new york', 'sunnyvale, california', 'cambridge, massachusetts', 'ozone park, new york','jackson, mississippi', 'south orange, new jersey','atlanta, georgia', 'madrid, spain', 'port costa, california','nicasio, california', 'bellingham, washington',
                       'woodacre, california', 'boston, massachusetts','longwood, florida', 'westlake, california','granite bay, california', 'campbell, california',
                       'santa ana, california', 'santa rosa, california', 'nevada city, california', 'providence, rhode island','stockton, california', 'vancouver, british columbia, canada','pacheco, california', 'irvine, california',
                       'kansas city, missouri', 'kassel, germany','stinson beach, california', 'philadelphia, pennsylvania', 'amsterdam, netherlands', 'napa, california', 'austin, texas','brooklyn, new york', 'bonaduz, switzerland',
                       'salt lake city, utah', 'bayshore, california', 'south lake tahoe, california', 'vacaville, california', 'miami, florida', 'long beach, california', 'cincinnati, ohio','phoenix, arizona', 'rochester, michigan', 'santa cruz, california',
                        'freedom, california', 'san quentin, california','utica, michigan', 'seaside, california', 'santa monica, california', 'woodbridge, virginia',
                        'arcadia, california', 'san antonio, texas', 'kula, hawaii','washington, district of columbia', 'muir beach, california','canyon, california', 'minneapolis, minnesota','san diego, california'], 
                    'not_san_francisco', inplace=True)  


In [43]:
df.offspring.value_counts()  
#Simplifying catagory into has kids, does not have kids, and unknown 

unknown_if_kids    17789
no_kids            11264
has_kids            3091
Name: offspring, dtype: int64

In [42]:
df['offspring'] = df['offspring'].fillna('unknown_if_kids') 
df.offspring.replace(['has a kid', 'has kids', "has kids, but doesn't want more", "has a kid, but doesn't want more",
                      'has kids, and might want more', 'has a kid, and might want more', 'has a kid, and wants more', 
                      'has kids, and wants more'
                     ],'has_kids', inplace=True) 
df.offspring.replace(["doesn't have kids, but might want them","doesn't have kids, but wants them", "doesn't have kids",
                      'might want kids', "doesn't have kids, and doesn't want any", "doesn't want kids",'wants kids',
                     ],'no_kids', inplace=True)

In [45]:
df.pets.value_counts() 
#Simplified categories

likes_dogs_and_cats          13651
unknown_pet_feelings          8473
likes_dogs                    8206
likes_cats                    1610
dislikes_cats_and_or_dogs      204
Name: pets, dtype: int64

In [44]:
df['pets'] = df['pets'].fillna('unknown_pet_feelings') 
df.pets.replace(['likes dogs and likes cats', 'likes dogs and has cats','has dogs and likes cats','has dogs and has cats', 
                     ],'likes_dogs_and_cats', inplace=True) 
df.pets.replace(['likes dogs and dislikes cats','has dogs','likes dogs','has dogs and dislikes cats',
                     ],'likes_dogs', inplace=True) 
df.pets.replace(['likes cats','has cats','dislikes dogs and likes cats', 'dislikes dogs and has cats'
                     ],'likes_cats', inplace=True) 
df.pets.replace(['dislikes cats','dislikes dogs and dislikes cats','dislikes dogs'
                     ],'dislikes_cats_and_or_dogs', inplace=True)

In [47]:
df.ethnicity.value_counts() 
#Grouped up those in the majority groups to include those that were mixed with one of the majority groups 

white                      18482
asian                       4363
ethnicity_other/unknown     4099
hispanic/latin              2628
black                       1487
indian                       713
middle_eastern               372
Name: ethnicity, dtype: int64

In [46]:
df['ethnicity'] = df['ethnicity'].fillna('ethnicity_other/unknown') 
df.loc[df['ethnicity'].str.contains('asian'), 'ethnicity'] = 'asian' 
df.loc[df['ethnicity'].str.contains('hispanic'), 'ethnicity'] = 'hispanic/latin' 
df.loc[df['ethnicity'].str.contains('black'), 'ethnicity'] = 'black' 
df.loc[df['ethnicity'].str.contains('middle eastern'), 'ethnicity'] = 'middle_eastern' 
df.loc[df['ethnicity'].str.contains('indian'), 'ethnicity'] = 'indian' 
df.ethnicity.replace(['pacific islander, white','pacific islander', 'native american, white', 'native american, white, other', 
                      'white, other','other', 'native american', 'pacific islander, other', 'pacific islander, white, other',
                      'native american, other', 'native american, pacific islander, white', 
                      'native american, pacific islander, white, other', 'native american, pacific islander'],'ethnicity_other/unknown', inplace=True)

In [49]:
df.religion.value_counts() 
#simplified catagories by removing atitudes 

religion_other/unknown    13248
agnosticism                5374
atheism                    3710
christianity               3319
catholicism                3038
judaism                    1851
buddhism                   1231
hinduism                    308
islam                        65
Name: religion, dtype: int64

In [48]:
df['religion'] = df['religion'].fillna('religion_other/unknown') 
df.loc[df['religion'].str.contains('agnosticism'), 'religion'] = 'agnosticism' 
df.loc[df['religion'].str.contains('catholicism'), 'religion'] = 'catholicism' 
df.loc[df['religion'].str.contains('christianity'), 'religion'] = 'christianity' 
df.loc[df['religion'].str.contains('atheism'), 'religion'] = 'atheism' 
df.loc[df['religion'].str.contains('judaism'), 'religion'] = 'judaism' 
df.loc[df['religion'].str.contains('buddhism'), 'religion'] = 'buddhism' 
df.loc[df['religion'].str.contains('hinduism'), 'religion'] = 'hinduism'
df.loc[df['religion'].str.contains('other'), 'religion'] = 'religion_other/unknown' 
df.loc[df['religion'].str.contains('islam'), 'religion'] = 'islam' 

In [51]:
df.sign.value_counts() 
#simplified signs

gemini         2884
leo            2879
cancer         2796
libra          2772
scorpio        2762
taurus         2713
virgo          2696
aries          2608
pisces         2600
aquarius       2562
sagittarius    2534
capricorn      2338
Name: sign, dtype: int64

In [50]:
df.loc[df['sign'].str.contains('aries'), 'sign'] = 'aries'
df.loc[df['sign'].str.contains('taurus'), 'sign'] = 'taurus'
df.loc[df['sign'].str.contains('gemini'), 'sign'] = 'gemini'
df.loc[df['sign'].str.contains('cancer'), 'sign'] = 'cancer'
df.loc[df['sign'].str.contains('leo'), 'sign'] = 'leo'
df.loc[df['sign'].str.contains('virgo'), 'sign'] = 'virgo'
df.loc[df['sign'].str.contains('libra'), 'sign'] = 'libra'
df.loc[df['sign'].str.contains('scorpio'), 'sign'] = 'scorpio'
df.loc[df['sign'].str.contains('sagittarius'), 'sign'] = 'sagittarius' 
df.loc[df['sign'].str.contains('capricorn'), 'sign'] = 'capricorn'
df.loc[df['sign'].str.contains('aquarius'), 'sign'] = 'aquarius'
df.loc[df['sign'].str.contains('pisces'), 'sign'] = 'pisces'

In [53]:
df.smokes.value_counts() 
#Simplified catagories 

does_not_smoke       24454
smokes                5587
unknown_if_smokes     2103
Name: smokes, dtype: int64

In [52]:
df['smokes'] = df['smokes'].fillna('unknown_if_smokes') 
df.smokes.replace(['no'],'does_not_smoke', inplace=True) 
df.smokes.replace(['sometimes','when drinking','yes','trying to quit'],'smokes', inplace=True)

In [54]:
df.age.value_counts()

27    1966
28    1936
26    1914
29    1863
30    1819
25    1722
31    1587
32    1535
24    1465
33    1324
34    1132
35    1091
23    1054
36     973
37     888
38     852
39     708
22     666
42     659
40     648
41     632
43     520
44     467
45     406
46     377
47     338
21     331
48     297
49     286
50     258
20     229
52     219
51     208
55     169
54     169
56     163
57     154
53     146
58     117
59     115
60     112
61      99
19      92
62      90
63      72
64      53
65      50
66      46
18      46
67      35
68      32
69      14
Name: age, dtype: int64

In [55]:
df.to_csv('processed_data.csv')